- drop some features
- go back to robust scaler
- keep 3 ensemble

In [1]:
!pip install --no-index --find-links /kaggle/input/pytorchtabnet/pytorch_tabnet-2.0.0-py3-none-any.whl pytorch-tabnet

Looking in links: /kaggle/input/pytorchtabnet/pytorch_tabnet-2.0.0-py3-none-any.whl
Processing /kaggle/input/pytorchtabnet/pytorch_tabnet-2.0.0-py3-none-any.whl


In [2]:
import os
import sys
import random
import warnings
import numpy as np
import pandas as pd 
from sklearn import preprocessing
from sklearn.metrics import log_loss,roc_auc_score
from sklearn.decomposition import PCA
from tqdm import tqdm_notebook as tqdm
from sklearn.cluster import KMeans
from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_selection import VarianceThreshold

from pytorch_tabnet.metrics import Metric
from pytorch_tabnet.tab_model import TabNetRegressor

sys.path.append('../input/multilabelstraifier/')
from ml_stratifiers import MultilabelStratifiedKFold
warnings.filterwarnings('ignore')

import time
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import tensorflow as tf

In [3]:
DATA_DIR = '/kaggle/input/lish-moa/'
train = pd.read_csv(DATA_DIR + 'train_features.csv')
targets = pd.read_csv(DATA_DIR + 'train_targets_scored.csv')
non_targets = pd.read_csv(DATA_DIR + 'train_targets_nonscored.csv')
test = pd.read_csv(DATA_DIR + 'test_features.csv')
sub = pd.read_csv(DATA_DIR + 'sample_submission.csv')

In [4]:
target_feats = [ i for i in targets.columns if i != "sig_id"]
g_feats = [i for i in train.columns if "g-" in i]
c_feats = [i for i in train.columns if "c-" in i]

noncons_train_index = train[train.cp_type=="ctl_vehicle"].index
cons_train_index = train[~train.index.isin(noncons_train_index)].index
noncons_test_index = test[test.cp_type=="ctl_vehicle"].index
cons_test_index = test[test.cp_type!="ctl_vehicle"].index

# preprocess

In [5]:
train = train[train.index.isin(cons_train_index)].copy().reset_index(drop=True)
targets = targets[targets.index.isin(cons_train_index)].copy().reset_index(drop=True)

# feature engineering

In [6]:
X = train.iloc[:,4:].copy().values
select = VarianceThreshold(threshold=0.7)
X_new = select.fit_transform(X)
drop_feats = list(np.array(train.iloc[:,4:].columns)[select.get_support()==False])
len(drop_feats)

33

In [7]:
train.drop(drop_feats, axis=1, inplace=True)
test.drop(drop_feats, axis=1, inplace=True)

In [8]:
#n_clusters_g = 15
#n_clusters_c = 5
#def create_cluster(train, test, kind, n_clusters):
#    if kind == "g":
#        train_ = train[g_feats].copy()
#        test_ = test[g_feats].copy()
#    else:
#        train_ = train[c_feats].copy()
#        test_ = test[c_feats].copy()    
#    kmeans = KMeans(n_clusters = n_clusters, random_state = 0).fit(train_)
#    train[f'clusters_{kind}'] = kmeans.labels_
#    test[f'clusters_{kind}'] = kmeans.predict(test_)
#    train = pd.get_dummies(train, columns = [f'clusters_{kind}'])
#    test = pd.get_dummies(test, columns = [f'clusters_{kind}'])
#    return train, test
    
#train, test = create_cluster(train, test, kind = 'g', n_clusters = n_clusters_g)
#train, test = create_cluster(train, test, kind = 'c', n_clusters = n_clusters_c)

In [9]:
#num = 10
#pca_c_cols = ["pca-c"+str(i+1) for i in range(num)]
#pca = PCA(n_components=num,random_state=42)
#c_train = pca.fit_transform(train[c_feats])
#c_test = pca.transform(test[c_feats])
#c_train = pd.DataFrame(c_train, columns=pca_c_cols)
#c_test = pd.DataFrame(c_test, columns=pca_c_cols)

#num = 60
#pca_g_cols = ["pca-g"+str(i+1) for i in range(num)]
#pca = PCA(n_components=num, random_state=42)
#g_train = pca.fit_transform(train[g_feats])
#g_test = pca.transform(test[g_feats])
#g_train = pd.DataFrame(g_train, columns=pca_g_cols)
#g_test = pd.DataFrame(g_test, columns=pca_g_cols)

#train = pd.concat([train, c_train],axis=1)
#test = pd.concat([test, c_test],axis=1)
#train = pd.concat([train, g_train],axis=1)
#test = pd.concat([test, g_test],axis=1)

In [10]:
def fe(df):
    tmp = df.copy()
    tmp.loc[:, 'cp_dose'] = tmp.loc[:, 'cp_dose'].map({'D1': 0, 'D2': 1})        
    tmp.drop(["cp_type", "sig_id"], axis=1, inplace=True)
    return tmp

train = fe(train)
test = fe(test)

print(train.shape, test.shape)

(21948, 841) (3982, 841)


In [11]:
fn_train = train.copy().to_numpy()
fn_test = test.copy().to_numpy()

ss = preprocessing.RobustScaler()
fn_train= ss.fit_transform(fn_train)
fn_test = ss.transform(fn_test)

fn_targets = targets.drop("sig_id", axis=1).copy().to_numpy()

In [12]:
fn_train.shape, fn_test.shape, fn_targets.shape, fn_non_targets.shape

NameError: name 'fn_non_targets' is not defined

# modelling

In [13]:
class LogitsLogLoss(Metric):
    def __init__(self):
        self._name = "logits_ll"
        self._maximize = False

    def __call__(self, y_true, y_pred):
        """
        Compute LogLoss of predictions.

        Parameters
        ----------
        y_true: np.ndarray
            Target matrix or vector
        y_score: np.ndarray
            Score matrix or vector

        Returns
        -------
            float
            LogLoss of predictions vs targets.
        """
        logits = 1 / (1 + np.exp(-y_pred))
        
        #smoothing = 0.001
        #n_classes = 206
        #y_true = y_true * (1 - smoothing) + np.ones_like(y_true) * smoothing / n_classes
        
        aux = (1-y_true)*np.log(1-logits+1e-15) + y_true*np.log(logits+1e-15)
        return np.mean(-aux)

In [14]:
MAX_EPOCH=200

smoothing = 0.001
p_min = smoothing
p_max = 1 - smoothing

def seed_everything(seed_value):
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    os.environ['PYTHONHASHSEED'] = str(seed_value)
    
    if torch.cuda.is_available(): 
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

def modelling_tabnet(tr, target, te, sample_seed):
    seed_everything(sample_seed) 
    tabnet_params = dict(n_d=12, n_a=12, n_steps=1, gamma=1.3, seed = sample_seed,
                     lambda_sparse=0, optimizer_fn=torch.optim.Adam,
                     optimizer_params=dict(lr=2e-2, weight_decay=1e-5),
                     mask_type='entmax',
                     scheduler_params=dict(mode="min",
                                           patience=5,
                                           min_lr=1e-5,
                                           factor=0.9,),
                     scheduler_fn=torch.optim.lr_scheduler.ReduceLROnPlateau,
                     verbose=10,
                     )
    test_cv_preds = []

    NB_SPLITS = 5
    mskf = MultilabelStratifiedKFold(n_splits=NB_SPLITS, random_state=0, shuffle=True)
    oof_preds = np.zeros([len(tr),target.shape[1]])
    scores = []
    for fold_nb, (train_idx, val_idx) in enumerate(mskf.split(train, target)):
        print("FOLDS : ", fold_nb+1)

        ## model
        X_train, y_train = tr[train_idx, :], target[train_idx, :]
        X_val, y_val = tr[val_idx, :], target[val_idx, :]
        model = TabNetRegressor(**tabnet_params)
    
        model.fit(X_train=X_train,
              y_train=y_train,
              eval_set=[(X_val, y_val)],
              eval_name = ["val"],
              eval_metric = ["logits_ll"],
              max_epochs=MAX_EPOCH,
              patience=20, batch_size=1024, virtual_batch_size=128,
              num_workers=1, drop_last=False,
              # use binary cross entropy as this is not a regression problem
              loss_fn=torch.nn.functional.binary_cross_entropy_with_logits)
    
        preds_val = model.predict(X_val)
        # Apply sigmoid to the predictions
        preds =  1 / (1 + np.exp(-preds_val))
        score = np.min(model.history["val_logits_ll"])
        oof_preds[val_idx,:] = preds
        #oof_preds[val_idx,:] = np.clip(preds, p_min, p_max)
        scores.append(score)

        # preds on test
        preds_test = model.predict(te)
        test_cv_preds.append(1 / (1 + np.exp(-preds_test)))
        #test_cv_preds.append(np.clip(1 / (1 + np.exp(-preds_test))), p_min, p_max)

    test_preds_all = np.stack(test_cv_preds)
    return oof_preds, test_preds_all

target_oof = np.zeros([len(fn_train),fn_targets.shape[1]])
target_pred = np.zeros([len(fn_test),fn_targets.shape[1]])

seeds = [0,1,2]
for seed_ in seeds:
    oof_preds, test_preds_all = modelling_tabnet(fn_train, fn_targets, fn_test, seed_)
    target_oof += oof_preds / len(seeds)
    target_pred += test_preds_all.mean(axis=0) / len(seeds)

FOLDS :  1
Device used : cuda
epoch 0  | loss: 0.57216 | val_logits_ll: 0.3141  |  0:00:02s
epoch 10 | loss: 0.02077 | val_logits_ll: 0.02065 |  0:00:11s
epoch 20 | loss: 0.01907 | val_logits_ll: 0.01906 |  0:00:22s
epoch 30 | loss: 0.01758 | val_logits_ll: 0.01815 |  0:00:33s
epoch 40 | loss: 0.01699 | val_logits_ll: 0.01749 |  0:00:43s
epoch 50 | loss: 0.01663 | val_logits_ll: 0.02037 |  0:00:53s
epoch 60 | loss: 0.0163  | val_logits_ll: 0.0173  |  0:01:03s
epoch 70 | loss: 0.01623 | val_logits_ll: 0.01701 |  0:01:13s
epoch 80 | loss: 0.01588 | val_logits_ll: 0.01703 |  0:01:23s
epoch 90 | loss: 0.0157  | val_logits_ll: 0.01696 |  0:01:34s
epoch 100| loss: 0.01545 | val_logits_ll: 0.01763 |  0:01:44s

Early stopping occured at epoch 104 with best_epoch = 84 and best_val_logits_ll = 0.01689
Best weights from best epoch are automatically used!
FOLDS :  2
Device used : cuda
epoch 0  | loss: 0.56393 | val_logits_ll: 0.28258 |  0:00:00s
epoch 10 | loss: 0.02086 | val_logits_ll: 0.0206  | 

In [15]:
t = pd.read_csv(DATA_DIR + 'train_targets_scored.csv')
train_checkscore = t.copy()
train_checkscore.loc[train_checkscore.index.isin(cons_train_index),target_feats] = target_oof
train_checkscore.loc[train_checkscore.index.isin(noncons_train_index),target_feats] = 0

t.drop("sig_id", axis=1, inplace=True)
print('OOF log loss: ', log_loss(np.ravel(t), np.ravel(np.array(train_checkscore.iloc[:,1:]))))

OOF log loss:  0.015206826543588205


In [16]:
sub[target_feats] = target_pred
sub.loc[noncons_test_index,target_feats] = 0
sub.to_csv('submission.csv', index=False)